In [1]:
import re,os
import numpy as np
import random
mypath="authors"

In [2]:
def dataSet(dirName):
    authors=[]
    path=os.getcwd()+'/'+dirName+'/'
    mylist=[]
    for filename in os.listdir(path):
        authors.append(filename)
        for file in os.listdir(path+filename):
            files=open(path+filename+'/'+file,'r',encoding="utf8")
            text=files.read().split()
            mylist.append(text) 
    return mylist,authors

In [3]:
dataset,authors=dataSet(mypath)
print(authors)

['Andres platonov', 'Maksim Gorki', 'Gabriel Garcia Marquez', 'James Joyce', 'Yasar Kemal', 'Oguz Atay', 'Ahmet hamdi']


In [4]:
docData=[]
docAuthors=[]
defaultDocWord=530
for i in range(len(authors)):
    lenBook=len(dataset[i])
    lenDoc=int(lenBook/defaultDocWord)
    if(lenBook/defaultDocWord!=0):
        lenDoc=lenDoc+1
    for k in range(lenDoc):
        docData.append(dataset[i][k*defaultDocWord:(k+1)*defaultDocWord])
        docAuthors.append(authors[i]) 


In [5]:
print(len(docData[0]))

530


In [6]:

def divideDataSet(docData,docAuthors):
   
    suffl = list(zip(docData, docAuthors))
    random.shuffle(suffl)
    docData, docAuthors = zip(*suffl)

    lenTrainSet=int(len(docData)*0.8)
    
    trainSetData=docData[0:lenTrainSet]
    trainSetAuthors=docAuthors[0:lenTrainSet]
    
    testSetData=docData[lenTrainSet:len(docData)]
    testSetAuthors=docAuthors[lenTrainSet:len(docAuthors)]
    return trainSetData,trainSetAuthors,testSetData,testSetAuthors


In [7]:
trainSetData,trainSetAuthors,testSetData,testSetAuthors = divideDataSet(docData,docAuthors)

In [8]:
def feature_extraction(docData):
   
        lenAllWord=len("".join(docData))
        lenDoc=len(docData)
        wordAverage=lenAllWord/lenDoc
        sentence=" ".join(docData)
        x=re.sub(r"[^\w\d'\s]+",'',sentence)
        
        
        uzunluk=len(''.join(x.split(" ")))
     
        countPunc=lenAllWord-uzunluk
        puncAverage=countPunc/len(docData)
        
        distTrainSetData=len(list(set(docData)))
        freqAverage=len(docData)/distTrainSetData
        
        features_value=[]
        features_value.append(wordAverage)
        features_value.append(puncAverage)
        features_value.append(freqAverage)

        return features_value
   

In [9]:

features_value= feature_extraction(trainSetData[0])

In [10]:
#print(features_value)


In [11]:
from math import sqrt

def vector_norm(features_value):
    norm = sqrt((features_value[0]**2)+(features_value[1]**2)+(features_value[2]**2))
    return norm

In [12]:
#print(vector_norm(features_value))

In [13]:

def authAverage():
    featureCount=len(feature_extraction(trainSetData[0]))
    
    auth_average=[]

    for auth in authors:
        index=[]
        features_average=np.zeros(featureCount)
        index = [index for index, value in enumerate(trainSetAuthors) if value == auth]
        for i in index:
            features_value=feature_extraction(trainSetData[i])
            for  k in range(featureCount):
                features_average[k]=features_average[k]+features_value[k]
        
        auth_average.append((features_average/len(index)).tolist())
 
   
    return auth_average

In [14]:
#authAverage()

In [15]:
def cosin(features_val1, features_val2):
    features_cos=0
   
    norm1=vector_norm(features_val1)
    norm2=vector_norm(features_val2)
    for i in range(len(features_val1)):
        features_cos= features_cos+(features_val1[i]*features_val2[i])
    features_cos=features_cos/(norm1*norm2)
    return features_cos

In [16]:
def cosfunc():
    lenAuthAverage=len(authAverage())
    auth_average=authAverage()
    lenTestSetData= len(testSetData)
    cos_list = np.zeros((lenTestSetData,lenAuthAverage, ))
    for k in range(lenAuthAverage):
        for i in range(lenTestSetData):

            test_feature=feature_extraction(testSetData[i])
            cos_list[i][k]=cosin(auth_average[k],test_feature)
    return cos_list

array([[ 0.99983528,  0.99948154,  0.9983838 , ...,  0.9993786 ,
         0.99978042,  0.99954678],
       [ 0.99978099,  0.99959284,  0.99809815, ...,  0.99934266,
         0.99974973,  0.99942913],
       [ 0.9999525 ,  0.99985916,  0.99896699, ...,  0.99992037,
         0.99998767,  0.99985123],
       ..., 
       [ 0.99916781,  0.9978403 ,  0.99965802, ...,  0.99887977,
         0.99901106,  0.99945251],
       [ 0.99989691,  0.99976686,  0.99923939, ...,  0.99999259,
         0.99994449,  0.99990531],
       [ 0.99990886,  0.99929233,  0.99956616, ...,  0.9997766 ,
         0.99986381,  0.99997724]])

In [55]:
def conf_matrix(cos_list):
    conf_mat=np.zeros((len(authors),len(authors)))
    for i in range(len(cos_list)):
        max_in=np.argmax(cos_list[i])
        #print(authors[max_in]," : ", testSetAuthors[i])
        conf_mat[authors.index(testSetAuthors[i])][max_in]=conf_mat[authors.index(testSetAuthors[i])][max_in]+1

    return conf_mat

In [56]:
matrix=conf_matrix(cosfunc())

In [57]:
def  pre_rec(matrix,i):

    colm_pred=sum(matrix[:,i])
    row_pred=sum(matrix[i])
    prec=matrix[i][i]/colm_pred
    recall=matrix[i][i]/row_pred

    return prec,recall

In [63]:
def f1_score():
    prec_average=0
    recall_average=0
    lenAuthors=len(authors)
    for i in range(lenAuthors):
        prec,recall=pre_rec(matrix,i)
        prec_average=prec_average+prec/lenAuthors
        recall_average=recall_average+recall/lenAuthors
    f1_score=2*(prec_average*recall_average)/(prec_average+recall_average)
    return f1_score

In [64]:
print(f1_score())

0.457505505782


In [61]:
def accuracy(matrix):
    matrix=conf_matrix(cosfunc())
    all_pred=sum(sum(matrix))
    t_pred=sum(np.diagonal(matrix))
    return (t_pred/all_pred)

In [62]:
# import re
# print(re.sub(r"[^\w\d'\s]+",'',sentence)) # noktalama işaretlerini çıkarır.;


In [22]:
# import time
# t0 = time.time()

# ortalama=len("".join(trainSetData[0]))/len(trainSetData[0])
# t1 = time.time()
# print("t: ", t1-t0)